In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  

# Configurações
FILE_PATH = r"_Case_Hotel.xlsx"          
PREFERRED_SHEET = "Base de Dados"        
OUT_XLSX = "hotel_base_limpa.xlsx"

In [2]:
# Conjunto mínimo de nomes de coluna esperados
EXPECTED_COLS = {
    "reserva_cancelada","tempo_antecedencia","data_chegada","ano_chegada",
    "mes_chegada","diadomes_chegada","nro_noites_fds","nro_noites_dds",
    "adultos","criancas","bebes","alimentacao","país","pais","segmento_mercado",
    "cliente_recorrente","tipo_quarto","tipo_pgto","agencia_turismo",
    "receita_por_noite","pedidos_especiais","status_reserva","data_status_reserva"
}


In [3]:
def choose_sheet(xls, preferred_name):
    #Escolhe a aba: tenta a preferida (case-insensitive), depois alguma com 'base', senão a 2ª, senão a 1ª
    names = xls.sheet_names
    for n in names:
        if n.strip().lower() == preferred_name.strip().lower():
            return n
    for n in names:
        if "base" in n.strip().lower():
            return n
    return names[1] if len(names) > 1 else names[0]


In [4]:
# 1) Utilitários

def detect_header_row(sample_df, max_rows=40):
    """
    Detecta a linha do cabeçalho procurando por >=3 nomes esperados na mesma linha.
    Usa as primeiras 'max_rows' linhas para procurar.
    """
    max_rows = min(max_rows, len(sample_df))
    expected_lower = {c.lower() for c in EXPECTED_COLS}
    for i in range(max_rows):
        row_vals = sample_df.iloc[i].astype(str).str.strip().str.lower().tolist()
        matches = sum(1 for c in expected_lower if c in row_vals)
        if matches >= 3:
            return i
    return 0  


In [5]:
def to_number(series):

    # Converte textos numéricos BR para float: Remove pontos de milhar; Troca vírgula por ponto; "NULL"/"None"/"nan" -> NaN; Erros -> NaN

    if series.dtype.kind in "biufc":
        return pd.to_numeric(series, errors="coerce")
    s = (
        series.astype(str)
        .str.strip()
        .replace({"NULL": np.nan, "None": np.nan, "nan": np.nan, "NaN": np.nan})
    )
    s = s.str.replace(".", "", regex=False).str.replace(",", ".", regex=False)
    return pd.to_numeric(s, errors="coerce")

In [6]:
def clean_text(series):
    # Normaliza textos: trim, colapsa espaços, converte nulos, Title-case simples
    return (
        series.astype(str)
        .str.strip()
        .replace({"NULL": np.nan, "None": np.nan, "nan": np.nan, "NaN": np.nan})
        .str.replace(r"\s+", " ", regex=True)
        .str.title()
    )

In [7]:
# Leitura robusta
xls = pd.ExcelFile(FILE_PATH)
print("Abas encontradas:", xls.sheet_names)
sheet = choose_sheet(xls, PREFERRED_SHEET)
print("Aba selecionada:", sheet)

# Lê amostra sem cabeçalho para detectar a linha do header
sample = pd.read_excel(FILE_PATH, sheet_name=sheet, header=None, nrows=50)
header_row = detect_header_row(sample)
print("Linha detectada como cabeçalho (0-based):", header_row)

# Lê a aba com o cabeçalho correto
df = pd.read_excel(FILE_PATH, sheet_name=sheet, header=header_row)
linhas_iniciais = len(df)

# Remove colunas e linhas completamente vazias
df = df.dropna(axis=1, how="all").dropna(axis=0, how="all").copy()

Abas encontradas: ['Dicionário Candidato', 'Base de Dados']
Aba selecionada: Base de Dados
Linha detectada como cabeçalho (0-based): 0


In [8]:
# Padronização de nomes de colunas
df.columns = [c.strip() for c in df.columns]

# Unifica 'pais' -> 'país' se for o caso
if "pais" in df.columns and "país" not in df.columns:
    df.rename(columns={"pais": "país"}, inplace=True)

print("\nDimensões do dataset bruto:", df.shape)
print("Colunas:", list(df.columns))


Dimensões do dataset bruto: (39859, 22)
Colunas: ['reserva_cancelada', 'tempo_antecedencia', 'data_chegada', 'ano_chegada', 'mes_chegada', 'diadomes_chegada', 'nro_noites_fds', 'nro_noites_dds', 'adultos', 'criancas', 'bebes', 'alimentacao', 'país', 'segmento_mercado', 'cliente_recorrente', 'tipo_quarto', 'tipo_pgto', 'agencia_turismo', 'receita_por_noite', 'pedidos_especiais', 'status_reserva', 'data_status_reserva']


In [9]:
#  Limpeza por tipo
#  Datas
# data_status_reserva: parse direto em formato BR
if "data_status_reserva" in df.columns:
    df["data_status_reserva"] = pd.to_datetime(
        df["data_status_reserva"], errors="coerce", dayfirst=True
    )

# data_chegada: reconstrução a partir de ano/mes/dia (evita casos 1969)
if {"data_chegada", "ano_chegada", "mes_chegada", "diadomes_chegada"}.issubset(df.columns):
    # Primeiro, tente converter o que já existe como texto (sem dayfirst para não gerar warning em ISO)
    parsed = pd.to_datetime(df["data_chegada"].astype(str), errors="coerce")
    # Reconstrói a partir de componentes
    reconstruida = pd.to_datetime(
        {
            "year":  df["ano_chegada"].astype("Int64"),
            "month": df["mes_chegada"].astype("Int64"),
            "day":   df["diadomes_chegada"].astype("Int64"),
        },
        errors="coerce",
    ).dt.normalize()
    # Usa reconstruída quando parsed for inválida ou com ano suspeito (<2015)
    mask_ruim = parsed.isna() | (parsed.dt.year < 2015)
    df["data_chegada"] = parsed.mask(mask_ruim, reconstruida)


In [10]:
# Numéricos inteiros (mantém nulos com Int64)
int_cols = [
    "tempo_antecedencia","nro_noites_fds","nro_noites_dds",
    "adultos","criancas","bebes","pedidos_especiais",
    "ano_chegada","mes_chegada","diadomes_chegada",
    "cliente_recorrente","reserva_cancelada","agencia_turismo"
]
for col in int_cols:
    if col in df.columns:
        df[col] = to_number(df[col]).round().astype("Int64")


In [11]:
# Numéricos decimais
if "receita_por_noite" in df.columns:
    df["receita_por_noite"] = to_number(df["receita_por_noite"])

# Textos / categorias
text_cols = ["alimentacao","segmento_mercado","tipo_quarto","tipo_pgto","status_reserva","país","pais"]
for col in text_cols:
    if col in df.columns:
        df[col] = clean_text(df[col])

# Unifica novamente 'pais' -> 'país' se necessário (após clean_text)
if "pais" in df.columns and "país" not in df.columns:
    df.rename(columns={"pais": "país"}, inplace=True)

In [12]:
# Boolean para análises
if "reserva_cancelada" in df.columns:
    df["reserva_cancelada_bool"] = df["reserva_cancelada"].astype("Int64").map({0: False, 1: True})

# Limpa literais de nulo que possam ter restado em objetos
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].replace({"NULL": np.nan, "None": np.nan, "nan": np.nan, "NaN": np.nan})

# País -> preencher NaN com "Desconhecido" (coluna é textual)
if "país" in df.columns:
    df["país"] = df["país"].fillna("Desconhecido")

In [13]:
# Nova coluna categórica para agência de turismo
if {"agencia_turismo", "segmento_mercado"}.issubset(df.columns):
    seg_lower = df["segmento_mercado"].astype(str).str.strip().str.lower()
    na_ag = df["agencia_turismo"].isna()

    df["agencia_turismo_cat"] = np.where(
        na_ag & (seg_lower == "direta"), "Reserva Direta",
        np.where(na_ag, "Não Informada", "Com Agência")
    )

In [14]:
# Remoção de duplicados e checagens
antes_dup = len(df)
df = df.drop_duplicates().reset_index(drop=True)  # garante aplicação em df final
depois_dup = len(df)

print("\nResumo de tipos:")
print(df.dtypes)

print("\nValores ausentes por coluna (top 20):")
print(df.isna().sum().sort_values(ascending=False).head(20))

print(f"\nLinhas iniciais (lidas): {linhas_iniciais}")
print(f"Linhas após limpeza e remoção de duplicados: {depois_dup}")
print(f"Duplicados removidos: {antes_dup - depois_dup}")

print("\nPrévia dos dados limpos:")
print(df.head(10))


Resumo de tipos:
reserva_cancelada                  Int64
tempo_antecedencia                 Int64
data_chegada              datetime64[ns]
ano_chegada                        Int64
mes_chegada                        Int64
diadomes_chegada                   Int64
nro_noites_fds                     Int64
nro_noites_dds                     Int64
adultos                            Int64
criancas                           Int64
bebes                              Int64
alimentacao                       object
país                              object
segmento_mercado                  object
cliente_recorrente                 Int64
tipo_quarto                       object
tipo_pgto                         object
agencia_turismo                    Int64
receita_por_noite                float64
pedidos_especiais                  Int64
status_reserva                    object
data_status_reserva       datetime64[ns]
reserva_cancelada_bool              bool
agencia_turismo_cat               objec

In [15]:
# Exportar dataset limpo
df.to_excel(OUT_XLSX, index=False)

print(f"\nArquivos salvos:")
print(f" - Excel: {OUT_XLSX}")


Arquivos salvos:
 - Excel: hotel_base_limpa.xlsx
